In [21]:
#import libraries
import pandas as pd
import numpy as np
import yfinance as yf
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings("ignore")



# Define the Transformer model
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = tf.keras.layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = tf.keras.layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(res)
    x = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

# Build and train the model
input_shape = (7,1)
head_size = 46
num_heads = 60
ff_dim = 55
num_transformer_blocks = 5
mlp_units = [256]
dropout = 0.14
mlp_dropout = 0.4

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    

    for _ in range(num_transformer_blocks):  # This is what stacks our transformer blocks
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = tf.keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)

    for dim in mlp_units:
        x = tf.keras.layers.Dense(dim, activation="relu")(x)
        x = tf.keras.layers.Dropout(mlp_dropout)(x)

    outputs = tf.keras.layers.Dense(1, activation="softmax")(x)  # this is a pass-through

    return tf.keras.Model(inputs, outputs)

# Define the learning rate scheduler
def lr_scheduler(epoch, lr, warmup_epochs=30, decay_epochs=100, initial_lr=1e-6, base_lr=1e-3, min_lr=5e-5):
    if epoch <= warmup_epochs:
        pct = epoch / warmup_epochs
        return ((base_lr - initial_lr) * pct) + initial_lr

    if epoch > warmup_epochs and epoch < warmup_epochs+decay_epochs:
        pct = 1 - ((epoch - warmup_epochs) / decay_epochs)
        return ((base_lr - min_lr) * pct) + min_lr

    return min_lr

def fetch_ticker_data(symbol, start_date, end_date):
    """Fetches stock data for a given symbol using yfinance."""
    ticker = yf.Ticker(symbol)
    data = ticker.history(start='2000-01-01', end=end_date)
    return data

def label_data(data):
    # Calculate the percentage change in price from one day to the next
    data['Percentage Change'] = data['Close'].pct_change()
    data['Percentage Change'] = data['Percentage Change'].shift(-1)
    data['Sentiment'] = pd.Series(np.where(data['Percentage Change'] > 0.025, 1, np.where(data['Percentage Change'] < -0.025, -1, 0)), index=data.index)
    # Drop any rows with missing values
    data.dropna(inplace=True)
    data.drop('Percentage Change',axis=1 , inplace=True)
    return data

def train_transformer(symbol_to_fetch,start_date ,end_date,no_model = True):
    #fetching data 
    stock = fetch_ticker_data(symbol_to_fetch, start_date, end_date)
    stock = stock.fillna(method="ffill", axis=0)
    stock = stock.fillna(method="bfill", axis=0)
    # stock.index = stock.index.date

    # Split the data into training and test sets
    train_data_index = np.searchsorted(stock.index.values, np.datetime64(start_date))
    train_data = stock.iloc[:train_data_index]
    test_data = stock.loc[start_date:]
    train_data = label_data(train_data)
    test_data = label_data(test_data)

    #trian & test data
    X_train_data = train_data.iloc[:,:-1]
    y_train_data = train_data.iloc[:,-1]
    X_test_data = test_data.iloc[:,:-1]
    y_test_data = test_data.iloc[:,-1]
    print(len(X_test_data))
    # Normalize the data
    normalizer = MinMaxScaler()
    X_train_data_normalizer = normalizer.fit_transform(X_train_data)
    X_test_data_normalizer = normalizer.transform(X_test_data)

    # # Reshape X_train_data_normalizer
    X_train = X_train_data_normalizer.reshape(X_train_data_normalizer.shape[0], X_train_data_normalizer.shape[1], 1)
    X_test = X_test_data_normalizer.reshape(X_test_data_normalizer.shape[0], X_test_data_normalizer.shape[1], 1)
    if no_model :
        model = build_model(
            input_shape,
            head_size=head_size,
            num_heads=num_heads,
            ff_dim=ff_dim,
            num_transformer_blocks=num_transformer_blocks,
            mlp_units=mlp_units,
            mlp_dropout=mlp_dropout,
            dropout=dropout,
        )

        model.compile(
            loss="mean_squared_error",
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            metrics=["mean_squared_error"],
        )

        callbacks = [
            tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
        ]

        # model.summary()
        history = model.fit(
            X_train,
            y_train_data,
            validation_split=0.2,
            epochs=100,
            batch_size=20,
            callbacks=callbacks,
        )
    return model,X_test,test_data
    #predictions
def preprae_sentiment_from_transformer(symbol_to_fetch,start_date,end_date):
    model,X_test,test_data = train_transformer(symbol_to_fetch = symbol_to_fetch, start_date = start_date, end_date = end_date)
    y_pred = model.predict(X_test) # this is the sentiment data 

    test_data['Sentiment'] = y_pred

    test_data.index = test_data.index.date
    test_data.to_csv('data/transformer_sentiment.csv')
    return test_data
    """next steps :  we need to additionally train the model if model is already present
    or take nearly 30 stocks and train the model with the huge data 
    or take every 5 mins data nad trian with it, and at last mix the test data with day wise"""

In [22]:
# Load stock price data
symbol_to_fetch = 'AAPL'
start_date = '2024-01-01'
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
result_data = preprae_sentiment_from_transformer(symbol_to_fetch, start_date, end_date)

85
Epoch 1/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 39s 77ms/step - loss: 1.1816 - mean_squared_error: 1.1816 - val_loss: 1.1465 - val_mean_squared_error: 1.1465 - learning_rate: 1.0000e-06
Epoch 2/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 16s 68ms/step - loss: 1.1810 - mean_squared_error: 1.1810 - val_loss: 1.1465 - val_mean_squared_error: 1.1465 - learning_rate: 3.4300e-05
Epoch 3/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 18s 73ms/step - loss: 1.2038 - mean_squared_error: 1.2038 - val_loss: 1.1465 - val_mean_squared_error: 1.1465 - learning_rate: 6.7600e-05
Epoch 4/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 15s 63ms/step - loss: 1.1958 - mean_squared_error: 1.1958 - val_loss: 1.1465 - val_mean_squared_error: 1.1465 - learning_rate: 1.0090e-04
Epoch 5/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 16s 64ms/step - loss: 1.1827 - mean_squared_error: 1.1827 - val_loss: 1.1465 - val_mean_squared_error: 1.1465 - learning_rate: 1.3420e-04
Epoch 6/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 17s 71ms/step - loss: 1.1667 - mean_squared_error: 1.1667 - v

In [3]:
import tensorflow as tf

model = tf.keras.models.load_model(filepath='models/transformer_MSFT_model.h5')


TypeError: too many positional arguments

In [4]:
import pandas as pd
df = pd.read_csv('data/transformer_sentiment.csv')
df.head()

,Unnamed: 0,Open,High,Low,Close,Volume,Dividends,Stock Splits,Sentiment,transformer_sentiment
0,2022-01-03,328.709004,331.306520,323.249301,328.120880,28865100,0.0,0.0,0,1.0
1,2022-01-04,328.199229,328.561926,319.661724,322.494507,32674300,0.0,0.0,-1,1.0
2,2022-01-05,319.406932,319.612795,309.722612,310.114685,40054300,0.0,0.0,0,1.0
3,2022-01-06,306.948600,312.388709,305.321470,307.664154,39646100,0.0,0.0,0,1.0
4,2022-01-07,307.928791,310.232259,303.949195,307.820984,32720000,0.0,0.0,0,1.0


In [6]:
df.loc['transformer_sentiment'] 

KeyError: 'transformer_sentiment'